# Short-term scheduling

Here we present the short-term scheduling module. The short-term scheduling module of spock has been concived to modify existing plan in order to add special or follow up observations, like transits or monitoring for instance.

<div class="alert alert-info">

**Note:** Before you insert new observations and upload them please make sure you have informed the SPECULOOS consortium 

</div>

## 1- Special target

<div class="alert alert-warning">
    
**WARNING:** First of all, users must ensure that they have entered requiered information of the target they wish to schedule either in <span style="color:teal"> *target_list_special.txt* </span> if it's a special observation or in <span style="color:teal"> *target_list_follow_up.txt* </span> if it's a transiting planet follow up.

</div>


### For a special observation with given start/end time 

Here is the existing **|night blocks|** for a given `date` and a given `telescope`:

In [1]:
import SPOCK.stats as SPOCKstats

SPOCKstats.read_night_plans_server(telescope='Europa',date='2020-12-12')

INFO:  Please add password.csv file in: /Users/elsaducrot/elsenv/lib/python3.7/site-packages/SPOCK-0.0.1-py3.7.egg/SPOCK/credentials/
INFO:  OK Password file exists


,target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
0,Trappist-1,2020-12-13 00:21:40.021,2020-12-13 02:43:20.021,141.666667,23.0,6.0,29.372136,-5.0,-2.0,-29.03172,"{'texp=50', ""filt=r'""}"
1,Sp0314+1603,2020-12-13 02:43:20.021,2020-12-13 04:31:40.020,108.333317,3.0,14.0,3.431808,16.0,3.0,5.47560,"{'filt=I+z', 'texp=70'}"
2,Sp0535-0931,2020-12-13 04:32:00.020,2020-12-13 08:50:00.020,258.000000,5.0,35.0,21.300600,-9.0,-31.0,-6.52332,"{'filt=I+z', 'texp=25'}"


First you want to create the **|night blocks|**. The only esstial information are: 

+ the name of the site, ex: `obs_name = 'SSO'`

+ the name of the telescope (because there can be several telescope per site), ex: `telescope ='Europa'`

+ the date the night starts, ex: `day_of_night ='2020-12-12 15:00:00'`

+ and the name of your target, `input_name = 'Trappist-1'`

+ start and end time in a list, ex: `start_end_range = ['2020-12-12 23:00:00','2020-12-13 01:00:00']`

<span style="color:teal">NB: you must first have filled info on the target in *target_list_special.txt* beforehand </span>

This is the block you wish to insert:

In [2]:
import SPOCK.short_term_scheduler as SPOCKST
from astropy.time import Time

schedule = SPOCKST.Schedules()

schedule.load_parameters()
schedule.day_of_night = Time('2020-12-12 15:00:00')
schedule.start_end_range = Time(['2020-12-12 23:00:00','2020-12-13 01:00:00'])
schedule.observatory_name = 'SSO'
schedule.telescope = 'Europa'
schedule.special_target_with_start_end(input_name="NGTS-11")



target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str7,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
NGTS-11,2020-12-12 23:00:00.018,2020-12-13 01:00:00.018,120.00000000000006,1.0,34.0,5.148000000001645,-14.0,-25.0,-8.936400000003175,"{""filt=g'"", 'texp=10'}"


Now you modify existing plan to insert your observation block:

In [3]:
schedule.make_scheduled_table()
schedule.planification()
schedule.make_night_block()

INFO:  Local path does not exist yet 
0
INFO:  situation 1
INFO:  situation 2
1
INFO:  situation 1
INFO:  situation 3, no change made to initial schedule
2
INFO:  situation 1
INFO:  situation 3, no change made to initial schedule
INFO:  no transition block


Here is how the modified **|night blocks|** looks like:

In [4]:
display(schedule.scheduled_table_sorted)

target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str11,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
NGTS-11,2020-12-13 00:21:24.626,2020-12-13 01:00:00.018,120.00000000000006,1.0,34.0,5.148000000001645,-14.0,-25.0,-8.936400000003175,"{""filt=g'"", 'texp=10'}"
Trappist-1,2020-12-13 01:00:00.018,2020-12-13 02:43:20.021,103.33338333333337,23.0,6.0,29.372136000018543,-5.0,-2.0,-29.031720000000334,"{'texp=50', ""filt=r'""}"
Sp0314+1603,2020-12-13 02:43:20.021,2020-12-13 04:31:40.020,108.3333166666666,3.0,14.0,3.4318080000036133,16.0,3.0,5.475599999978158,"{'filt=I+z', 'texp=70'}"
Sp0535-0931,2020-12-13 04:32:00.020,2020-12-13 08:50:00.020,258.00000000000006,5.0,35.0,21.300600000004266,-9.0,-31.0,-6.5233199999974545,"{'filt=I+z', 'texp=25'}"


### For a special target that you wish to observe as much as possible

Here is the existing **|night blocks|** for a given `date` and a given `telescope`:

In [5]:
import SPOCK.stats as SPOCKstats

SPOCKstats.read_night_plans_server(telescope='Artemis',date='2020-12-10')

,target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
0,NGTS-11,2020-12-10 19:05:40.009,2020-12-11 00:49:00.009,343.333333,1.0,34.0,5.148000,-14.0,-25.0,-8.93640,"{'filt=g', 'texp=10'}"
1,Sp0840+1824,2020-12-11 00:49:00.009,2020-12-11 06:53:20.009,364.333333,8.0,40.0,29.634552,18.0,24.0,8.72676,"{'texp=26', 'filt=I+z'}"


First you want to create the **|night blocks|**. The only essential information needed are: 

+ the name of the site, ex: `obs_name = 'SSO'`

+ the name of the telescope (because there can be several telescope per site), ex: `telescope = 'Europa'`

+ the date the night starts, ex: `day_of _night = '2020-12-12 15:00:00'`

+ and the name of your target, `input_name = 'Trappist-1'`

<span style="color:teal">NB: you must first have filled info on the target in *target_list_special.txt* beforehand </span>

In [6]:
import SPOCK.short_term_scheduler as SPOCKST
from astropy.time import Time

schedule = SPOCKST.Schedules()

schedule.load_parameters()
schedule.day_of_night = Time('2020-12-10 15:00:00')
schedule.observatory_name = 'SNO'
schedule.telescope = 'Artemis'
schedule.special_target(input_name="WASP-85Ab")



target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str9,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
WASP-85Ab,2020-12-11 03:05:00.027,2020-12-11 06:53:20.027,228.33333333333334,11.0,43.0,38.00820000000698,6.0,33.0,49.45220639999988,"{""filt=g'"", 'texp=10'}"


In [7]:
schedule.make_scheduled_table()
schedule.planification()
schedule.make_night_block()

INFO:  Local path does not exist yet 
0
INFO:  situation 7
INFO:  situation 10, no change made to initial schedule
1
INFO:  situation 6
INFO:  no transition block


Here is how the modified **|night blocks|** looks like:

In [8]:
display(schedule.scheduled_table_sorted)

target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str13,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
NGTS-11,2020-12-10 19:05:40.009,2020-12-11 03:05:00.027,479.33363333333324,1.0,34.0,5.148000000001645,-14.0,-25.0,-8.93639999999678,"{'filt=g', 'texp=10'}"
Sp0840+1824,2020-12-11 00:49:00.009,2020-12-11 03:05:00.027,136.00030000000004,8.0,40.0,29.634552000006664,18.0,24.0,8.726760000003821,"{'texp=26', 'filt=I+z'}"
WASP-85Ab,2020-12-11 03:05:00.027,2020-12-11 06:53:02.798,228.0461833333333,11.0,43.0,38.00820000000698,6.0,33.0,49.45220639999988,"{""filt=g'"", 'texp=10'}"


## 2- Follow-up

Here is the existing **|night blocks|** for a given `date` and a given `telescope`:

In [9]:
import SPOCK.stats as SPOCKstats

SPOCKstats.read_night_plans_server(telescope='Artemis',date='2020-09-15')

,target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
0,Sp0008+4918,2020-09-15 20:02:00.010,2020-09-16 06:00:40.010,598.666667,0.0,8.0,55.36284,49.0,18.0,56.3202,"{'filt=I+z', 'texp=20'}"


In [10]:
import SPOCK.short_term_scheduler as SPOCKST
from astropy.time import Time

schedule = SPOCKST.Schedules()

schedule.load_parameters()
schedule.day_of_night = Time('2020-09-15 15:00:00')
schedule.observatory_name = 'SNO'
schedule.telescope = 'Artemis'

schedule.transit_follow_up(name="Trappist-1b")

INFO:  Trappist-1b next transit: 
['2020-09-15 21:06:24.278']
INFO:  start_transit of  Trappist-1b  :  2020-09-15 19:49:57.067
INFO:  end_transit of  Trappist-1b  :  2020-09-15 22:22:53.490



target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str11,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
Trappist-1b,2020-09-15 20:40:20.012,2020-09-15 22:23:00.012,102.66666666666667,23.0,6.0,28.999920000019017,-5.0,-2.0,-27.99996000000064,"{'filt=I+z', 'texp=23'}"


In [11]:
schedule.make_scheduled_table()
schedule.planification()
schedule.make_night_block()

INFO:  Local path does not exist yet 
0
INFO:  situation 6
INFO:  no transition block


In [12]:
display(schedule.scheduled_table_sorted)

target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str13,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
Sp0008+4918,2020-09-15 20:02:00.010,2020-09-15 20:40:20.012,38.333366666666834,0.0,8.0,55.362840000000055,49.0,18.0,56.32019999999159,"{'filt=I+z', 'texp=20'}"
Trappist-1b,2020-09-15 20:40:20.012,2020-09-15 22:23:00.012,102.66666666666667,23.0,6.0,28.999920000019017,-5.0,-2.0,-27.99996000000064,"{'filt=I+z', 'texp=23'}"
Sp0008+4918_2,2020-09-15 22:23:00.012,2020-09-16 06:00:40.010,457.66663333333315,0.0,8.0,55.362840000000055,49.0,18.0,56.32019999999159,"{'filt=I+z', 'texp=20'}"


## 3- Save your plans and night blocks

If you are satisfied with this modified **|night blocks|** you can save it. Executing this cell will upadate your local spock database. The **|night blocks|** will be saved in `your_spock_path + "/DATABASE/telescope/Archive_night_blocks/"`

In [13]:
schedule = SPOCKST.Schedules()
schedule.day_of_night = Time('2020-09-15 15:00:00')
schedule.telescope = 'Artemis'

SPOCKST.save_schedule(save=True,over_write=True,day=schedule.day_of_night,telescope=schedule.telescope)


INFO:  "/Users/elsaducrot/spock_2/night_blocks_propositions/night_blocks_Artemis_2020-09-15.txt" has been over-written to "/Users/elsaducrot/spock_2/DATABASE/Artemis/"


Now that you have saved the night block it's time to create the ACP plans to send to the control PC of the corresonding telescope. To do so you just have to execute the cell below with the following information:

* `day`, start date you wish to make ACP plans for

* `nb_days`, number of days you wish to make plan for (starting at the date define in `day`)

* `telescope`, name of the telescope at sake

ACP plans will be saved in `your_spock_path + "/DATABASE/telescope/Plans_by_date/"`

In [14]:
SPOCKST.make_plans(day = schedule.day_of_night,nb_days=1,telescope = schedule.telescope)


Now you can upload these ACP plans and **|night blocks|** to the [Cambridge Archive](http://www.mrao.cam.ac.uk/SPECULOOS/) as well as the plans sent to the control PCs. You will just need to provide :

* `day`, start date you for which you wish to upload plans and **|night blocks|**

* `nb_days`, number of days you wish to upload (starting at the date define in `day`)

* `telescope`, name of the telescope at sake

In [15]:
SPOCKST.upload_plans(day=schedule.day_of_night, nb_days=1,telescope = schedule.telescope)

-----> 2020-09-15 Plans uploaded on the Cambridge server
-----> 2020-09-15 Night plans uploaded on the Cambridge server
-----> 2020-09-15 Zip Plans_by_dates folder uploaded on the Cambridge server
-----> 2020-09-15 Zip Plans_by_dates folder uploaded on the HUB for Artemis
INFO:  Path local 'Gant chart' =  /Users/elsaducrot/spock_2/SPOCK_Figures/Preview_schedule.html
INFO:  Path database = 'Gant chart' =  speculoos@appcs.ra.phy.cam.ac.uk:/appct/data/SPECULOOSPipeline/Preview_schedule.html
